## Table of Contents
0. Initial Setup
1. Utility Functions
2. Community Detection on Synthetic Networks
   - 2.1 Load Networks
   - 2.2 Community Detection Algorithms
   - 2.3 Evaluation Metrics
   - 2.4 Visualization
   - 2.5 Discussion
3. Conclusions

## 0. Initial Setup

In [9]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from collections import defaultdict
from collections import Counter
from cdlib import evaluation
import community as community_louvain
from tqdm import tqdm
from itertools import combinations
from networkx.algorithms.community import greedy_modularity_communities
from networkx.algorithms.community.quality import modularity

try:
    import infomap
except:
    !pip install infomap
    import infomap

plt.style.use('ggplot')
%matplotlib inline

REPORT_FOLDER = "report"
PLOTS_FOLDER = "plots"

for folder in [REPORT_FOLDER, PLOTS_FOLDER]:
    if not os.path.exists(folder):
        os.makedirs(folder)

## 1. Utility Functions

In [10]:
def load_network(path):
    """
    Load a Pajek-format network and convert to simple undirected NetworkX graph.

    Parameters:
        path (str): Path to .net file.

    Returns:
        G (networkx.Graph): Cleaned graph with integer-labeled nodes.
    """
    G = nx.read_pajek(path)
    G = nx.Graph(G)
    G = nx.convert_node_labels_to_integers(G)
    return G


def detect_communities_infomap(G):
    """
    Apply Infomap algorithm to detect communities in G.

    Parameters:
        G (networkx.Graph): Input graph.

    Returns:
        communities (list[set]): Detected communities.
    """
    im = infomap.Infomap()
    mapping = im.add_networkx_graph(G)
    im.run()
    modules = im.get_modules()
    community_dict = defaultdict(set)
    for node, com in modules.items():
        community_dict[com].add(mapping[node])
    return list(community_dict.values())


def detect_communities_louvain(G):
    """
    Apply Louvain algorithm to detect communities in G using NetworkX interface.

    Parameters:
        G (networkx.Graph): Input graph.

    Returns:
        communities (list[set]): Detected communities.
    """
    return list(nx.community.louvain_communities(G))


def detect_communities_greedy(G):
    """
    Apply Greedy modularity algorithm to detect communities in G.

    Parameters:
        G (networkx.Graph): Input graph.

    Returns:
        communities (list[set]): Detected communities.
    """
    return list(greedy_modularity_communities(G))


def evaluate_partitions(detected, ground_truth):
    """
    Compute evaluation metrics for a detected partition against ground truth.

    Parameters:
        detected (list[set]): Detected communities.
        ground_truth (list[set]): Ground-truth communities.

    Returns:
        dict: Scores for NMI, ARI, NVI.
    """
    return {
        "NMI": evaluation.normalized_mutual_information(detected, ground_truth).score,
        "ARI": evaluation.adjusted_rand_index(detected, ground_truth).score,
        "NVI": evaluation.normalized_variation_of_information(detected, ground_truth).score,
    }


def ground_truth_partition(N=300, block_size=60):
    """
    Generate the ground-truth partition from known SBM structure.

    Parameters:
        N (int): Total number of nodes.
        block_size (int): Size of each block.

    Returns:
        list[set]: Ground truth communities.
    """
    return [set(range(i * block_size, (i + 1) * block_size)) for i in range(N // block_size)]



## 2. Community Detection on Synthetic Networks

### 2.1 Load Networks
We load all synthetic networks from the folder `data/`, generated according to the Stochastic Block Model. Each network has 300 nodes and 5 blocks. The only parameter that changes is `prr`, ranging from 0.02 to 1.00.

Each filename has the form:
`synthetic_network_N_300_blocks_5_prr_{value}_prs_0.02.net`

In [11]:
from glob import glob
import re

networks = {}

for path in sorted(glob("data/*.net")):
    match = re.search(r"prr_([0-9.]+)_prs", path)
    if match:
        prr = float(match.group(1))
        networks[prr] = load_network(path)

networks = dict(sorted(networks.items()))  # Sort by prr


### 2.2 Community Detection Algorithms
We now apply Infomap, Louvain, and Greedy modularity maximization to each synthetic network.

In [12]:
communities_infomap = {}
communities_louvain = {}
communities_greedy = {}
modularity_infomap = {}
modularity_louvain = {}
modularity_greedy = {}

for prr, G in tqdm(networks.items(), desc="Detecting communities"):
    communities_infomap[prr] = detect_communities_infomap(G)
    communities_louvain[prr] = detect_communities_louvain(G)
    communities_greedy[prr] = detect_communities_greedy(G)

    modularity_infomap[prr] = modularity(G, communities_infomap[prr])
    modularity_louvain[prr] = modularity(G, communities_louvain[prr])
    modularity_greedy[prr] = modularity(G, communities_greedy[prr])


Detecting communities:   0%|          | 0/50 [00:00<?, ?it/s]

. Found 2 levels with codelength 6.478899126

=> Trial 1/1 finished in 0.005032415s with codelength 6.47889913


Summary after 1 trial
Best end modular solution in 2 levels:
Per level number of modules:         [          5,           0] (sum: 5)
Per level number of leaf nodes:      [          0,         300] (sum: 300)
Per level average child degree:      [          5,          60] (average: 59.0984)
Per level codelength for modules:    [0.176429994, 0.000000000] (sum: 0.176429994)
Per level codelength for leaf nodes: [0.000000000, 6.302469132] (sum: 6.302469132)
Per level codelength total:          [0.176429994, 6.302469132] (sum: 6.478899126)

  Infomap ends at 2025-04-11 20:50:50
  (Elapsed time: 0.010798547s)
  Infomap v2.8.0 starts at 2025-04-11 20:59:17
  -> Input network: 
  -> No file output!
  OpenMP 201511 detected with 12 threads...
  -> Ordinary network input, using the Map Equation for first order network flows
Calculating global network flow using flow model 'undirected'

Detecting communities:   2%|▏         | 1/50 [00:00<00:08,  5.91it/s]

. Found 2 levels with codelength 7.768460593

=> Trial 1/1 finished in 0.011550588s with codelength 7.76846059


Summary after 1 trial
Best end modular solution in 2 levels:
Per level number of modules:         [         23,           0] (sum: 23)
Per level number of leaf nodes:      [          0,         300] (sum: 300)
Per level average child degree:      [         23,     13.0435] (average: 13.7525)
Per level codelength for modules:    [2.397355467, 0.000000000] (sum: 2.397355467)
Per level codelength for leaf nodes: [0.000000000, 5.371105125] (sum: 5.371105125)
Per level codelength total:          [2.397355467, 5.371105125] (sum: 7.768460593)

  Infomap ends at 2025-04-11 20:59:17
  (Elapsed time: 0.016711793s)
  Infomap v2.8.0 starts at 2025-04-11 20:59:17
  -> Input network: 
  -> No file output!
  OpenMP 201511 detected with 12 threads...
  -> Ordinary network input, using the Map Equation for first order network flows
Calculating global network flow using flow model 'undirected

Detecting communities:   4%|▍         | 2/50 [00:00<00:08,  5.49it/s]

...

Detecting communities:  96%|█████████▌| 48/50 [00:15<00:00,  2.38it/s]

. Found 2 levels with codelength 6.501445918

=> Trial 1/1 finished in 0.004382394s with codelength 6.50144592


Summary after 1 trial
Best end modular solution in 2 levels:
Per level number of modules:         [          5,           0] (sum: 5)
Per level number of leaf nodes:      [          0,         300] (sum: 300)
Per level average child degree:      [          5,          60] (average: 59.0984)
Per level codelength for modules:    [0.185216718, 0.000000000] (sum: 0.185216718)
Per level codelength for leaf nodes: [0.000000000, 6.316229200] (sum: 6.316229200)
Per level codelength total:          [0.185216718, 6.316229200] (sum: 6.501445918)

  Infomap ends at 2025-04-11 20:59:32
  (Elapsed time: 0.010364985s)
  Infomap v2.8.0 starts at 2025-04-11 20:59:33
  -> Input network: 
  -> No file output!
  OpenMP 201511 detected with 12 threads...
  -> Ordinary network input, using the Map Equation for first order network flows
Calculating global network flow using flow model 'undirected'

Detecting communities:  98%|█████████▊| 49/50 [00:16<00:00,  2.48it/s]

. Found 2 levels with codelength 6.492159075

=> Trial 1/1 finished in 0.004176627s with codelength 6.49215907


Summary after 1 trial
Best end modular solution in 2 levels:
Per level number of modules:         [          5,           0] (sum: 5)
Per level number of leaf nodes:      [          0,         300] (sum: 300)
Per level average child degree:      [          5,          60] (average: 59.0984)
Per level codelength for modules:    [0.181589656, 0.000000000] (sum: 0.181589656)
Per level codelength for leaf nodes: [0.000000000, 6.310569419] (sum: 6.310569419)
Per level codelength total:          [0.181589656, 6.310569419] (sum: 6.492159075)

  Infomap ends at 2025-04-11 20:59:33
  (Elapsed time: 0.006596312s)
  Infomap v2.8.0 starts at 2025-04-11 20:59:33
  -> Input network: 
  -> No file output!
  OpenMP 201511 detected with 12 threads...
  -> Ordinary network input, using the Map Equation for first order network flows
Calculating global network flow using flow model 'undirected'

Detecting communities: 100%|██████████| 50/50 [00:16<00:00,  3.03it/s]


### 2.3 Evaluation Metrics
We now compare the detected partitions with the ground truth using the following metrics:
- Normalized Mutual Information (NMI)
- Adjusted Rand Index (ARI)
- Normalized Variation of Information (NVI)

In [13]:
evaluation_scores_infomap = {}
evaluation_scores_louvain = {}
evaluation_scores_greedy = {}

ground_truth = ground_truth_partition()

for prr, G in tqdm(networks.items(), desc="Evaluating partitions"):
    evaluation_scores_infomap[prr] = evaluate_partitions(communities_infomap[prr], ground_truth)
    evaluation_scores_louvain[prr] = evaluate_partitions(communities_louvain[prr], ground_truth)
    evaluation_scores_greedy[prr] = evaluate_partitions(communities_greedy[prr], ground_truth)

Evaluating partitions:   0%|          | 0/50 [00:00<?, ?it/s]


AttributeError: 'list' object has no attribute 'communities'